## Arguments

In [ ]:
model_name = "mistralai/Mistral-7B-Instruct-v0.1"
# model_cache = "INSERT" # TODO
reading_vec_dataset_name = "sycophancy_function_facts"
eval_dataset_name = "anthropic_evals_nlp"

## Dependencies

In [ ]:
import dotenv
import sys
import os

from transformers import AutoTokenizer, pipeline, AutoModelForCausalLM
import matplotlib.pyplot as plt
import torch
from tqdm import tqdm
import numpy as np

from repe import repe_pipeline_registry # TODO: install into env, ensure using common and up-to-date version
repe_pipeline_registry()

In [ ]:
# Datasets


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

## Load Model

In [ ]:
model_name_or_path = "mistralai/Mistral-7B-Instruct-v0.1"

model = AutoModelForCausalLM.from_pretrained(model_name_or_path, torch_dtype=torch.float16, device_map="auto")
use_fast_tokenizer = "LlamaForCausalLM" not in model.config.architectures
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=use_fast_tokenizer, padding_side="left", legacy=False)
tokenizer.pad_token_id = 0 

## Get Reading Vector

In [ ]:
rep_token = -1
hidden_layers = list(range(-1, -model.config.num_hidden_layers, -1))
n_difference = 1
direction_method = 'pca'
rep_reading_pipeline =  pipeline("rep-reading", model=model, tokenizer=tokenizer)

In [ ]:
user_tag = "[INST]"
assistant_tag = "[/INST]"

data_path = "../../data/facts/facts_true_false.csv"
dataset = sycophancy_function_dataset(data_path, tokenizer, user_tag, assistant_tag)

In [ ]:
honesty_rep_reader = rep_reading_pipeline.get_directions(
    dataset['train']['data'], 
    rep_token=rep_token, 
    hidden_layers=hidden_layers, 
    n_difference=n_difference, 
    train_labels=dataset['train']['labels'], 
    direction_method=direction_method,
    batch_size=32,
)

## Evals

### Evaluate normal model